In [ ]:
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 829.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install triton

In [ ]:
def extract_answer(prompt):
    answer_marker = '### Answer:\n'
    # end_marker = ''

    start_pos = prompt.find(answer_marker) + len(answer_marker)
    # end_pos = prompt.find(end_marker, start_pos)

    answer = prompt[start_pos:len(prompt)].strip()
    return answer

In [ ]:
from datasets import load_dataset
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the context given

### Context:
{}

### Question:
{}

### Answer:
"""
def formatting_prompts_func(examples):
    context=examples['context']
    q       = examples["question"]
    texts = []
    for context, q in zip(context, q):
        text = prompt.format(context, q)
        texts.append(text)
    return { "text" : texts, }
pass
dataset = load_dataset("squad_v2", split = "validation")
dataset = dataset.map(formatting_prompts_func, batched = True,)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
if True:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Qwen2-7B", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [ ]:
preds=[]
for i in range (0,len(dataset["text"])):
  inputs = tokenizer(
            [
              dataset["text"][i]
            ], return_tensors = "pt").to("cuda")  # Add batch dimension
  print('Round: ',i)
  output = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)
  ans=extract_answer(tokenizer.batch_decode(output, skip_special_tokens=True)[0])
  if(ans.find('Cannot Find Answer')!=-1):
    ans=''
    prob=1.
  else:
    prob=0.
  temp={'prediction_text': ans, 'id':[dataset[i]['id']] , 'no_answer_probability': prob }
  preds.append(temp)
references = []

for i in range (0,len(dataset['text'])):
  print('Round Ref: ',i)
  temp={'answers': dataset[i]['answers'], 'id':[dataset[i]['id']] }
  references.append(temp)


Streaming output truncated to the last 5000 lines.
Round Ref:  6873
Round Ref:  6874
Round Ref:  6875
Round Ref:  6876
Round Ref:  6877
Round Ref:  6878
Round Ref:  6879
Round Ref:  6880
Round Ref:  6881
Round Ref:  6882
Round Ref:  6883
Round Ref:  6884
Round Ref:  6885
Round Ref:  6886
Round Ref:  6887
Round Ref:  6888
Round Ref:  6889
Round Ref:  6890
Round Ref:  6891
Round Ref:  6892
Round Ref:  6893
Round Ref:  6894
Round Ref:  6895
Round Ref:  6896
Round Ref:  6897
Round Ref:  6898
Round Ref:  6899
Round Ref:  6900
Round Ref:  6901
Round Ref:  6902
Round Ref:  6903
Round Ref:  6904
Round Ref:  6905
Round Ref:  6906
Round Ref:  6907
Round Ref:  6908
Round Ref:  6909
Round Ref:  6910
Round Ref:  6911
Round Ref:  6912
Round Ref:  6913
Round Ref:  6914
Round Ref:  6915
Round Ref:  6916
Round Ref:  6917
Round Ref:  6918
Round Ref:  6919
Round Ref:  6920
Round Ref:  6921
Round Ref:  6922
Round Ref:  6923
Round Ref:  6924
Round Ref:  6925
Round Ref:  6926
Round Ref:  6927
Round Ref:  69

In [ ]:
import json
FILE_PATH = './predsbase.json'
with open(FILE_PATH, 'w') as output_file:
	json.dump(preds, output_file, indent=2)

In [ ]:
import json
FILE_PATH = './refsbase.json'
with open(FILE_PATH, 'w') as output_file:
	json.dump(references, output_file, indent=2)

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is 

In [ ]:
import evaluate

In [ ]:
metric = evaluate.load('squad_v2')
def adjust_prediction_format(predictions):
    for pred in predictions:
        if isinstance(pred['id'], list):
            pred['id'] = pred['id'][0]  # Convert list to single string
    return predictions


In [ ]:
import json
f = open('predsbase.json')
preds = json.load(f)
f.close()
f = open('refsbase.json')
references = json.load(f)
f.close()

In [ ]:
adjusted_predictions = adjust_prediction_format(preds)
adjref=adjust_prediction_format(references)
results = metric.compute(predictions=adjusted_predictions,references=adjref,no_answer_threshold=1.)
print(results)

{'exact': 0.32005390381537946, 'f1': 16.49260577296779, 'total': 11873, 'HasAns_exact': 0.6410256410256411, 'HasAns_f1': 33.03250815493363, 'HasAns_total': 5928, 'NoAns_exact': 0.0, 'NoAns_f1': 0.0, 'NoAns_total': 5945, 'best_exact': 50.07159100480081, 'best_exact_thresh': 0.0, 'best_f1': 50.091438841593664, 'best_f1_thresh': 0.0}
